In [1]:
# 必要なモジュールをインポートする。
from qiskit.optimization import QuadraticProgram
from qiskit import Aer
from qiskit import QuantumCircuit, execute
from qiskit.aqua.operators import I, X, Y
from qiskit.aqua.components.initial_states import Custom
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QAOA

/Users/harukikondo/opt/anaconda3/lib/python3.8/site-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')


In [2]:
# 回路を用意する。
qubo = QuadraticProgram()
# 変数を用意する。
qubo.binary_var('q0')
qubo.binary_var('q1')
qubo.binary_var('q2')
qubo.binary_var('q3')

In [3]:
# 辞書型でQUBOの係数を代入する。
linear = {'q0': 4, 'q1': 4, 'q2': 4, 'q3': 4}
quadratic = {('q0', 'q1'): 4, ('q0', 'q2'): 4, ('q1', 'q2'): 8, ('q1', 'q3'): 2, ('q2', 'q3'): 2}
qubo.minimize(linear=linear, quadratic=quadratic)
print(qubo)


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 4 q0 + 4 q1 + 4 q2 + 4 q3 + [ 8 q0*q1 + 8 q0*q2 + 16 q1*q2 + 4 q1*q3
      + 4 q2*q3 ]/2
Subject To

Bounds
 0 <= q0 <= 1
 0 <= q1 <= 1
 0 <= q2 <= 1
 0 <= q3 <= 1

Binaries
 q0 q1 q2 q3
End



In [4]:
# ミキサーを用意する。
mixer = ((X^X^I^I)+(Y^Y^I^I))/2 + ((I^I^X^X) + (I^I^Y^Y))/2
# 量子回路を用意する。
initial_state_circuit = QuantumCircuit(4)
# 各量子ゲートを配置する
initial_state_circuit.h(0)
initial_state_circuit.cx(0, 1)
initial_state_circuit.x(0)
initial_state_circuit.h(2)
initial_state_circuit.cx(2, 3)
initial_state_circuit.x(2)
# マシンを選択して実行する。
initial_state_vec = execute(initial_state_circuit, Aer.get_backend('statevector_simulator')).result().get_statevector()
# 初期状態を取得する。
initial_state = Custom(4, state_vector=initial_state_vec)

/Users/harukikondo/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/operators/operator_base.py:46: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.operators', 'qiskit.opflow', 'qiskit-terra')
/Users/harukikondo/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/components/initial_states/custom.py:79: DeprecationWarning: The Custom class is deprecated as of Aqua 0.9 and will be removed no earlier than 3 months after the release date. Instead, all algorithms and circuits accept a plain QuantumCircuit. Custom(state_vector=vector) is the same as a circuit where the ``initialize(vector/np.linalg.norm(vector))`` method has been called.
  super().__init__()


In [5]:
# マシンを選択して実行する。
quantum_instance = QuantumInstance(Aer.get_backend('statevector_simulator'))
# ステップ数
step = 1
# QAOAを実行する。
qaoa_mes = QAOA(quantum_instance=quantum_instance, p=step, initial_state=initial_state, mixer=mixer)
qaoa = MinimumEigenOptimizer(qaoa_mes)
qaoa_result = qaoa.solve(qubo)
# 結果を出力する。
print(qaoa_result)

/Users/harukikondo/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/quantum_instance.py:135: DeprecationWarning: The class qiskit.aqua.QuantumInstance is deprecated. It was moved/refactored to qiskit.utils.QuantumInstance (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_class('aqua.QuantumInstance',
/Users/harukikondo/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/components/optimizers/optimizer.py:49: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.components.optimizers',
/Users/harukikondo/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/algorithms/vq_algorithm.py:70: DeprecationWarning: The class qiskit.aqua.algorithms.VQAlgorithm is deprecat

optimal function value: 8.0
optimal value: [1. 0. 0. 1.]
status: SUCCESS


In [6]:
# ミキサーを使った場合
print('no mixer')
# マシンを選択する。
quantum_instance = QuantumInstance(Aer.get_backend('statevector_simulator'))
# ステップ数
step = 1
qaoa_mes = QAOA(quantum_instance=quantum_instance, p=step)
qaoa = MinimumEigenOptimizer(qaoa_mes)
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)


no mixer
optimal function value: 0.0
optimal value: [0. 0. 0. 0.]
status: SUCCESS
